In [1]:
import pandas as pd
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import folium
from pyproj import Proj
import io
from PIL import Image

In [2]:
df = pd.read_csv('Presas.csv')
df.head()

,Unnamed: 0,Nombre,Fase,Coordenadas
0,0,,,-
1,1,ACEBO,Explotación,0184037 - 4461097
2,2,ACEHUCHE,Explotación,0194081 - 4412473
3,3,"ACEÑA, LA",Explotación,0396755 - 4495825
4,4,ACIBAL / PONTILLON CASTRO,Explotación,0037937 - 4720650


In [3]:
df = df.drop(0)

In [4]:
df = df.drop("Unnamed: 0", axis=1)

In [5]:
df["Fase"].unique()

array([' Explotación', ' Puesta en Carga', ' Construcción'], dtype=object)

In [6]:
df["x"] = [i.split(" - ")[0] for i in df.Coordenadas]
df["y"] = [i.split(" - ")[1] for i in df.Coordenadas]

In [7]:
df["x"] = df["x"].astype(float)
df["y"] = df["y"].astype(float)

In [8]:
df = df.drop("Coordenadas", axis=1)

In [9]:
df

,Nombre,Fase,x,y
1,ACEBO,Explotación,184037.0,4461097.0
2,ACEHUCHE,Explotación,194081.0,4412473.0
3,"ACEÑA, LA",Explotación,396755.0,4495825.0
4,ACIBAL / PONTILLON CASTRO,Explotación,37937.0,4720650.0
5,ADEYAHAMEN,Explotación,-951179.0,3277854.0
...,...,...,...,...
1222,ZORITA DE LOS MOLINOS (AZUD),Construcción,356673.0,4513827.0
1223,ZUFRE,Explotación,210540.0,4188725.0
1224,ZUJAR,Explotación,285172.0,4310644.0
1225,ZUJAR (DIQUE DEL COLLADO),Explotación,285155.0,4310665.0


In [10]:
df["Fase"].value_counts()

 Explotación        1173
 Construcción         38
 Puesta en Carga      15
Name: Fase, dtype: int64

In [11]:
myProj = Proj("+proj=utm +zone=30, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [12]:
lon, lat = myProj(df['x'].values, df['y'].values, inverse=True)

In [13]:
df["x"] = lon
df["y"] = lat

In [14]:
df

,Nombre,Fase,x,y
1,ACEBO,Explotación,-6.714190,40.240727
2,ACEHUCHE,Explotación,-6.573429,39.807223
3,"ACEÑA, LA",Explotación,-4.220385,40.606797
4,ACIBAL / PONTILLON CASTRO,Explotación,-8.622740,42.500124
5,ADEYAHAMEN,Explotación,-17.782942,28.804680
...,...,...,...,...
1222,ZORITA DE LOS MOLINOS (AZUD),Construcción,-4.698107,40.762925
1223,ZUFRE,Explotación,-6.287552,37.800063
1224,ZUJAR,Explotación,-5.477922,38.918429
1225,ZUJAR (DIQUE DEL COLLADO),Explotación,-5.478124,38.918614


In [15]:
spainmap = folium.Map(location=[40.342827, -3.509423], zoom_start=6, tiles='Stamen Terrain')
spainmap

In [16]:
presas = folium.map.FeatureGroup()
fase = list(df["Fase"])
colores = {" Explotación":"blue", " Puesta en Carga":"green", " Construcción":"red"}
for lat, lng, fase in zip(df.y, df.x, fase): 
    presas.add_child(
    folium.Marker([lat, lng],
    icon=folium.Icon(icon='tint', color = colores[fase])
        )
    )
    

In [17]:
# Latitud = list(df.y)
# Longitud = list(df.x)
# labels = list(df.Nombre)

# for lat, lng, label in zip(Latitud, Longitud, labels):
#     folium.Marker([lat, lng], popup=label).add_to(presas)

In [18]:
spainmap.add_child(presas)

In [19]:
spainmap.save('Mapa de presas.html')